# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [12]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/divya/Desktop/python-api-challenge/python-api-challenge/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,msanga,-6.0500,36.0333,22.25,85,48,2.23,TZ,1712770041
1,1,iqaluit,63.7506,-68.5145,-1.15,63,75,1.54,CA,1712770041
2,2,hawaiian paradise park,19.5933,-154.9731,19.46,98,100,0.45,US,1712770041
3,3,blackmans bay,-43.0167,147.3167,10.29,86,75,0.45,AU,1712770042
4,4,keflavik,64.0049,-22.5624,6.95,65,75,8.23,IS,1712770042


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
!pip show geoviews

Name: geoviews
Version: 1.12.0
Summary: GeoViews is a Python library that makes it easy to explore and visualize geographical, meteorological, and oceanographic datasets, such as those used in weather, climate, and remote sensing research.
Home-page: https://geoviews.org
Author: 
Author-email: 
License: BSD 3-Clause
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: bokeh, cartopy, holoviews, numpy, packaging, panel, param, pyproj, shapely, xyzservices
Required-by: 


In [14]:
%%capture --no-display

# Configure the map plot
mapplot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    color='City',
    cmap='Set3',
    size="Humidity",
    tiles='OSM'
)

# Display the map plot
mapplot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & 
                                (city_data_df['Wind Speed'] < 4.5) & 
                                (city_data_df['Cloudiness'] == 0)]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.sample(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
268,268,artesia,33.8658,-118.0831,26.61,41,0,1.54,US,1712770094
46,46,tshabong,-26.0500,22.4500,22.72,33,0,2.87,BW,1712770050
424,424,callao,-12.0667,-77.1500,26.16,65,0,3.60,PE,1712769886
510,510,villa union,23.2000,-106.2333,23.98,46,0,0.00,MX,1712770144
240,240,kapa'a,22.0752,-159.3190,23.39,82,0,4.12,US,1712770088
210,210,al bawiti,28.3492,28.8659,21.66,31,0,3.97,EG,1712770083
165,165,lihue,21.9789,-159.3672,22.98,77,0,4.12,US,1712769805
131,131,siwa oasis,29.2041,25.5195,23.93,24,0,3.64,EG,1712770068
296,296,lompoc,34.6391,-120.4579,22.66,57,0,2.06,US,1712770100
155,155,dhangadhi,28.7079,80.5961,25.35,11,0,1.84,NP,1712770072


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
46,tshabong,BW,-26.0500,22.4500,33,
111,wailua homesteads,US,22.0669,-159.3780,82,
131,siwa oasis,EG,29.2041,25.5195,24,
155,dhangadhi,NP,28.7079,80.5961,11,
165,lihue,US,21.9789,-159.3672,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",  # Replace with the correct category ID for hotels
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat,lng = row["Lat"],row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()
    #print(name_address) #if, no hotels are found, remove first # to see why the response isn't working correctly

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
tshabong - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
siwa oasis - nearest hotel: Flower of the city hotel
dhangadhi - nearest hotel: Hotel Bikas
lihue - nearest hotel: Kauai Palms
tolanaro - nearest hotel: Hôtel Mahavokey
al bawiti - nearest hotel: Old Oasis Hotel
kapa'a - nearest hotel: Pono Kai Resort
artesia - nearest hotel: Days Inn & Suites by Wyndham Artesia
lompoc - nearest hotel: Red Roof Inn Lompoc
callao - nearest hotel: Casa Ronald
villa union - nearest hotel: No hotel found
port hueneme - nearest hotel: Holiday Inn Express Port Hueneme
santa ana - nearest hotel: Holiday Inn Express


,City,Country,Lat,Lng,Humidity,Hotel Name
46,tshabong,BW,-26.0500,22.4500,33,No hotel found
111,wailua homesteads,US,22.0669,-159.3780,82,Hilton Garden Inn Kauai Wailua Bay
131,siwa oasis,EG,29.2041,25.5195,24,Flower of the city hotel
155,dhangadhi,NP,28.7079,80.5961,11,Hotel Bikas
165,lihue,US,21.9789,-159.3672,77,Kauai Palms


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(x='Lng', y='Lat', geo=True, tiles='OSM', size='Humidity', height=400, width=600, hover_cols=['City', 'Hotel Name', 'Country'])

# Display the map
hotel_map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name,Country)